In [1]:
pip install edgedb  

Note: you may need to restart the kernel to use updated packages.


In [2]:
# need to install edgedb for this to work

In [2]:
import pandas as pd
import json
import re

import datetime
import time

import edgedb
#https://www.edgedb.com/docs/clients/python/usage

In [3]:
import numpy as np

In [4]:
#Basic connection testing
# Note * This File needs to be in the same folder as your edgedb init proj
client = edgedb.create_client()

In [5]:
embedded_df = pd.read_csv('embedded_recipes.csv')
seasonal_df = pd.read_csv('seasonal.csv')
spice_df = pd.read_csv('spice_df.csv')

In [6]:
embedded_df

,Unnamed: 0,RecipeId,Name,cleaned_ingre,combined,embeddings
0,0,39,Biryani,"saffron, milk, hot green chili pepper, onion, ...",Biryani is made up of the following ingredient...,"[-0.089964359998703, 0.14856766164302826, 0.21..."
1,1,41,Carina's-Tofu-Vegetable-Kebabs,"extra firm tofu, eggplant, zucchini, mushroom,...",Carina's-Tofu-Vegetable-Kebabs is made up of t...,"[-0.19750580191612244, -0.17205673456192017, 0..."
2,2,43,Best-Blackbottom-Pie,"graham cracker crumb, sugar, butter, sugar, co...",Best-Blackbottom-Pie is made up of the followi...,"[-0.13601799309253693, 0.2751603126525879, -0...."
3,3,45,Buttermilk-Pie-With-Gingersnap-Crumb-Crust,"sugar, margarine, egg, flour, salt, buttermilk...",Buttermilk-Pie-With-Gingersnap-Crumb-Crust is ...,"[-0.011905956082046032, 0.1964324414730072, -0..."
4,4,47,Butter-Pecan-Cookies,"butter, brown sugar, sugar, vanilla extract, f...",Butter-Pecan-Cookies is made up of the followi...,"[-0.41587966680526733, 0.3891988694667816, -0...."
...,...,...,...,...,...,...
460744,460744,538941,Belgian-Fries-With-Andalouse-Sauce,"mayonnaise, tomato paste, red onion, green bel...",Belgian-Fries-With-Andalouse-Sauce is made up ...,"[0.08309848606586456, -0.03274141252040863, -0..."
460745,460745,538970,Creamy-Tuscan-Garlic-Chicken,"chicken breast, olive oil, heavy cream, chicke...",Creamy-Tuscan-Garlic-Chicken is made up of the...,"[0.0787457674741745, 0.10412728041410446, -0.3..."
460746,460746,538991,Nif's-Easy-Korean-Beef-and-Rice,"lean ground beef, garlic clove, brown sugar, s...",Nif's-Easy-Korean-Beef-and-Rice is made up of ...,"[-0.2255922108888626, 0.05001573637127876, 0.1..."
460747,460747,539090,Cuban-Mojo-Potatoes,"yukon gold potato, salt, lime, olive oil, red ...",Cuban-Mojo-Potatoes is made up of the followin...,"[0.09820163995027542, -0.2252044975757599, -0...."


In [6]:
capstone_df = pd.read_csv('captsone_df.csv') # already ran this skip

In [8]:
capstone_df

,Unnamed: 0,RecipeId,Name,TotalTime,Description,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,...,ingredients,Calories_PerServing,FatContent_PerServing,SaturatedFatContent_PerServing,CholesterolContent_PerServing,SodiumContent_PerServing,CarbohydrateContent_PerServing,FiberContent_PerServing,SugarContent_PerServing,ProteinContent_PerServing
0,0,39,Biryani,265,Make and share this Biryani recipe from Food.com.,Chicken Breast,"['Chicken Thigh & Leg', 'Chicken', 'Poultry', ...","c(""1"", ""4"", ""2"", ""2"", ""8"", ""1/4"", ""8"", ""1/2"", ...","['saffron', 'milk', 'hot green chili peppers',...",3.0,...,"['saffron', 'milk', 'hot green chili pepper', ...",185.116667,9.800000,2.766667,62.133333,61.400,14.066667,1.500000,3.400000,10.566667
1,1,41,Carina's Tofu-Vegetable Kebabs,1460,This dish is best prepared a day in advance to...,Soy/Tofu,"['Beans', 'Vegetable', 'Low Cholesterol', 'Wee...","c(""12"", ""1"", ""2"", ""1"", ""10"", ""1"", ""3"", ""2"", ""2...","['extra firm tofu', 'eggplant', 'zucchini', 'm...",4.5,...,"['extra firm tofu', 'eggplant', 'zucchini', 'm...",268.050000,12.000000,1.900000,0.000000,779.300,32.100000,8.650000,16.050000,14.650000
2,2,43,Best Blackbottom Pie,140,Make and share this Best Blackbottom Pie recip...,Pie,"['Dessert', 'Weeknight', 'Stove Top', '< 4 Hou...","c(""1 1/4"", ""1/4"", ""6"", ""1/3"", ""1/4"", ""1/4"", ""2...","['graham cracker crumbs', 'sugar', 'butter', '...",1.0,...,"['graham cracker crumb', 'sugar', 'butter', 's...",54.737500,2.412500,1.362500,11.787500,33.450,7.250000,0.225000,5.312500,0.875000
3,3,45,Buttermilk Pie With Gingersnap Crumb Crust,80,Make and share this Buttermilk Pie With Ginger...,Pie,"['Dessert', 'Healthy', 'Weeknight', 'Oven', '<...","c(""3/4"", ""1"", ""1"", ""2"", ""3"", ""1/4"", ""1"", ""1/2""...","['sugar', 'margarine', 'egg', 'flour', 'salt',...",4.0,...,"['sugar', 'margarine', 'egg', 'flour', 'salt',...",28.500000,0.887500,0.212500,3.062500,35.225,4.687500,0.062500,3.087500,0.525000
4,4,47,Butter Pecan Cookies,64,Make and share this Butter Pecan Cookies recip...,Dessert,"['Cookie & Brownie', 'Fruit', 'Nuts', 'Weeknig...","c(""3/4"", ""1/2"", ""1"", ""1"", ""1"", ""2"", ""1"")","['butter', 'brown sugar', 'granulated sugar', ...",4.0,...,"['butter', 'brown sugar', 'sugar', 'vanilla ex...",69.000000,5.600000,1.400000,6.300000,15.000,4.500000,0.600000,1.600000,0.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460744,460744,538941,Belgian Fries With Andalouse Sauce,40,Belgian fries are cooked twice for a soft midd...,Potato,"['Vegetable', '< 60 Mins']","c(""1"", ""2"", ""2"", ""1"", ""1"", ""1"", ""1/4"", ""3"", ""4"")","['mayonnaise', 'tomato paste', 'red onions', '...",5.0,...,"['mayonnaise', 'tomato paste', 'red onion', 'g...",290.966667,28.833333,4.000000,0.000000,31.200,8.266667,1.066667,0.533333,0.966667
460745,460745,538970,Creamy Tuscan Garlic Chicken,25,This is a quick recipe worthy of company. My ...,European,"['< 30 Mins', 'Easy']","c(""1 1/2"", ""2"", ""1"", ""1/2"", ""1"", ""1"", ""1/2"", ""...","['boneless skinless chicken breasts', 'olive o...",5.0,...,"['chicken breast', 'olive oil', 'heavy cream',...",134.875000,9.300000,4.475000,50.375000,163.125,1.725000,0.275000,0.725000,11.025000
460746,460746,538991,Nif's Easy Korean Beef and Rice,20,This is an easy go-to staple in our house! Spi...,Meat,"['Korean', 'Asian', 'Healthy', '< 30 Mins', 'E...","c(""1 1/2"", ""5"", ""1 - 1 1/2"", ""1/2 - 3/4"", ""1/...","['lean ground beef', 'garlic cloves', 'brown s...",5.0,...,"['lean ground beef', 'garlic clove', 'brown su...",1166.900000,21.600000,7.700000,110.600000,1466.900,186.800000,3.800000,27.900000,50.300000
460747,460747,539090,Cuban Mojo Potatoes,60,Make and share this Cuban Mojo Potatoes recipe...,Cuban,"['Caribbean', 'Low Protein', 'Low Cholesterol'...","c(""1 1/2"", ""1"", ""1/2"", ""1/4"", ""1"", ""6"", ""3"", ""...","['yukon gold potatoes', 'salt', 'lime', 'olive.

In [9]:
capstone_df.columns

Index(['Unnamed: 0', 'RecipeId', 'Name', 'TotalTime', 'Description',
       'RecipeCategory', 'Keywords', 'RecipeIngredientQuantities',
       'RecipeIngredientParts', 'AggregatedRating', 'ReviewCount', 'Calories',
       'FatContent', 'SaturatedFatContent', 'CholesterolContent',
       'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent',
       'ProteinContent', 'RecipeInstructions', 'Popularity', 'ingredient_num',
       'ingredients', 'Calories_PerServing', 'FatContent_PerServing',
       'SaturatedFatContent_PerServing', 'CholesterolContent_PerServing',
       'SodiumContent_PerServing', 'CarbohydrateContent_PerServing',
       'FiberContent_PerServing', 'SugarContent_PerServing',
       'ProteinContent_PerServing'],
      dtype='object')

In [11]:
DataToLoad = capstone_df[['RecipeId', 'Name', 'TotalTime', 'Description',
       'RecipeCategory', 'Keywords', 'Calories',
       'FatContent', 'SaturatedFatContent', 'CholesterolContent',
       'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent',
       'ProteinContent', 'RecipeInstructions', 'Popularity', 'ingredient_num',
       'ingredients', 'Calories_PerServing', 'FatContent_PerServing',
       'SaturatedFatContent_PerServing', 'CholesterolContent_PerServing',
       'SodiumContent_PerServing', 'CarbohydrateContent_PerServing',
       'FiberContent_PerServing', 'SugarContent_PerServing',
       'ProteinContent_PerServing']]

In [7]:
embedded_df.columns

Index(['Unnamed: 0', 'RecipeId', 'Name', 'cleaned_ingre', 'combined',
       'embeddings'],
      dtype='object')

In [8]:
seasonal_df.columns

Index(['Unnamed: 0', 'Ingredient', 'Growing Season'], dtype='object')

In [9]:
spice_df.columns

Index(['Unnamed: 0', 'Spice', 'Description'], dtype='object')

In [10]:
EmbeddedToLoad = embedded_df[['RecipeId', 'Name', 'cleaned_ingre', 'combined',
       'embeddings']]

In [11]:
SeasonalToLoad = seasonal_df[['Ingredient', 'Growing Season']]
SpiceToLoad = spice_df[['Spice', 'Description']]

In [12]:
# Approach 1: Using np.array_split()
num_splits = 3
smaller_dfs_array_split = np.array_split(EmbeddedToLoad, num_splits)

# Calculate the approximate number of rows for each smaller DataFrame
num_rows = len(EmbeddedToLoad)
rows_per_df = num_rows // num_splits

# Approach 1: Using np.array_split()
smaller_dfs_array_split = np.array_split(EmbeddedToLoad, num_splits)

# Approach 2: Using DataFrame slicing
smaller_df_1 = EmbeddedToLoad[:rows_per_df]
smaller_df_2 = EmbeddedToLoad[rows_per_df: 2 * rows_per_df]
smaller_df_3 = EmbeddedToLoad[2 * rows_per_df:]

# Approach 3: Using np.random.rand()
random_array = np.random.rand(len(EmbeddedToLoad))

smaller_df_1 = EmbeddedToLoad[random_array < 1/3]
smaller_df_2 = EmbeddedToLoad[(random_array >= 1/3) & (random_array < 2/3)]
smaller_df_3 = EmbeddedToLoad[random_array >= 2/3]

In [15]:
# smaller_df_1 # this was a check

In [13]:
DataToLoad1 = smaller_df_1
DataToLoad2 = smaller_df_2
DataToLoad3 = smaller_df_3

In [14]:
DataToLoad1.to_json('DataToLoad1.json', orient="records")
DataToLoad2.to_json('DataToLoad2.json', orient="records")
DataToLoad3.to_json('DataToLoad3.json', orient="records")

In [15]:
with open('DataToLoad1.json', "r") as f:
    jsonData1 = json.load(f)
    jsonData1str = json.dumps(jsonData1)
    

In [16]:
with open('DataToLoad2.json', "r") as f:
    jsonData2 = json.load(f)
    jsonData2str = json.dumps(jsonData2)
    

In [17]:
with open('DataToLoad3.json', "r") as f:
    jsonData3 = json.load(f)
    jsonData3str = json.dumps(jsonData3)
    

In [18]:
import ast

In [21]:
# Assuming you have a JSON data string named 'jsonData1str'
data = json.loads(jsonData1str)

# Split the data into smaller batches
batch_size = 100  # Choose an appropriate batch size
num_batches = len(data) // batch_size + (1 if len(data) % batch_size > 0 else 0)

for batch_num in range(num_batches):
    start_idx = batch_num * batch_size
    end_idx = (batch_num + 1) * batch_size
    batch_data = data[start_idx:end_idx]

    # Convert embeddings values using ast.literal_eval()
    for item in batch_data:
        item['embeddings'] = ast.literal_eval(item['embeddings'])

    # Convert the batch data back to a JSON string
    batch_data_str = json.dumps(batch_data)

    # Perform the insertion for the current batch
    client.query("""
        WITH raw_data := <json>$data
        FOR item IN json_array_unpack(raw_data) UNION (
            INSERT RecipeEmbedding {
                Name := <str>item['Name'],
                RecipeId := <int32>item['RecipeId'],
                cleaned_ingre := <str>item['cleaned_ingre'],
                combined := <str>item['combined'],
                embeddings := <v100>item['embeddings']
            }
        );
    """, data=batch_data_str)

In [22]:
# Assuming you have a JSON data string named 'jsonData1str'
data = json.loads(jsonData2str)

# Split the data into smaller batches
batch_size = 100  # Choose an appropriate batch size
num_batches = len(data) // batch_size + (1 if len(data) % batch_size > 0 else 0)

for batch_num in range(num_batches):
    start_idx = batch_num * batch_size
    end_idx = (batch_num + 1) * batch_size
    batch_data = data[start_idx:end_idx]

    # Convert embeddings values using ast.literal_eval()
    for item in batch_data:
        item['embeddings'] = ast.literal_eval(item['embeddings'])

    # Convert the batch data back to a JSON string
    batch_data_str = json.dumps(batch_data)

    # Perform the insertion for the current batch
    client.query("""
        WITH raw_data := <json>$data
        FOR item IN json_array_unpack(raw_data) UNION (
            INSERT RecipeEmbedding {
                Name := <str>item['Name'],
                RecipeId := <int32>item['RecipeId'],
                cleaned_ingre := <str>item['cleaned_ingre'],
                combined := <str>item['combined'],
                embeddings := <v100>item['embeddings']
            }
        );
    """, data=batch_data_str)

In [23]:
# Assuming you have a JSON data string named 'jsonData1str'
data = json.loads(jsonData3str)

# Split the data into smaller batches
batch_size = 100  # Choose an appropriate batch size
num_batches = len(data) // batch_size + (1 if len(data) % batch_size > 0 else 0)

for batch_num in range(num_batches):
    start_idx = batch_num * batch_size
    end_idx = (batch_num + 1) * batch_size
    batch_data = data[start_idx:end_idx]

    # Convert embeddings values using ast.literal_eval()
    for item in batch_data:
        item['embeddings'] = ast.literal_eval(item['embeddings'])

    # Convert the batch data back to a JSON string
    batch_data_str = json.dumps(batch_data)

    # Perform the insertion for the current batch
    client.query("""
        WITH raw_data := <json>$data
        FOR item IN json_array_unpack(raw_data) UNION (
            INSERT RecipeEmbedding {
                Name := <str>item['Name'],
                RecipeId := <int32>item['RecipeId'],
                cleaned_ingre := <str>item['cleaned_ingre'],
                combined := <str>item['combined'],
                embeddings := <v100>item['embeddings']
            }
        );
    """, data=batch_data_str)

In [19]:
# Get the data types of each column
column_data_types = DataToLoad.dtypes

In [20]:
column_data_types

RecipeId                            int64
Name                               object
TotalTime                           int64
Description                        object
RecipeCategory                     object
Keywords                           object
Calories                          float64
FatContent                        float64
SaturatedFatContent               float64
CholesterolContent                float64
SodiumContent                     float64
CarbohydrateContent               float64
FiberContent                      float64
SugarContent                      float64
ProteinContent                    float64
RecipeInstructions                 object
Popularity                         object
ingredient_num                      int64
ingredients                        object
Calories_PerServing               float64
FatContent_PerServing             float64
SaturatedFatContent_PerServing    float64
CholesterolContent_PerServing     float64
SodiumContent_PerServing          

In [5]:
reviews_df = pd.read_csv('reviews.csv')

In [36]:
reviews_df

,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified
0,2,992,2008,gayg msft,5,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,7,4384,1634,Bill Hilbrich,4,"I cut back on the mayo, and made up the differ...",2001-10-17T16:49:59Z,2001-10-17T16:49:59Z
2,9,4523,2046,Gay Gilmore ckpt,2,i think i did something wrong because i could ...,2000-02-25T09:00:00Z,2000-02-25T09:00:00Z
3,13,7435,1773,Malarkey Test,5,easily the best i have ever had. juicy flavor...,2000-03-13T21:15:00Z,2000-03-13T21:15:00Z
4,14,44,2085,Tony Small,5,An excellent dish.,2000-03-28T12:51:00Z,2000-03-28T12:51:00Z
...,...,...,...,...,...,...,...,...
1401977,2090339,139499,2002080368,terrylbiggs,2,I was disappointed. I couldn't wait to make th...,2020-12-27T23:57:54Z,2020-12-27T23:57:54Z
1401978,2090340,148484,41805321,rogerberry,5,Nothing to drain. And I don’t heat up the liqu...,2020-12-28T00:44:42Z,2020-12-28T00:44:42Z
1401979,2090341,264191,2002901848,Reiketsukan 6.,5,Good base recipe for someone to start with. I ...,2020-12-28T01:04:43Z,2020-12-28T01:04:43Z
1401980,2090345,411791,2002901938,Sue M.,5,Thank you so much for this amazing recipe! I l...,2020-12-28T03:07:10Z,2020-12-28T03:07:10Z


In [39]:
# Approach 1: Using np.array_split()
num_splits = 3
smaller_dfs_array_split = np.array_split(reviews_df, num_splits)


# smaller_dfs_array_split is a list containing the three smaller DataFrames
smaller_df_1 = smaller_dfs_array_split[0]
smaller_df_2 = smaller_dfs_array_split[1]
smaller_df_3 = smaller_dfs_array_split[2]

# Approach 2: Using DataFrame slicing
# Calculate the approximate number of rows for each smaller DataFrame
num_rows = len(reviews_df)
rows_per_df = num_rows // num_splits

smaller_df_1 = reviews_df[:rows_per_df]
smaller_df_2 = reviews_df[rows_per_df: 2 * rows_per_df]
smaller_df_3 = reviews_df[2 * rows_per_df:]

# Approach 3: Using np.random.rand()
# Create a random array with the same length as the original DataFrame
random_array = np.random.rand(len(reviews_df))

# Assign rows to each smaller DataFrame based on the random array values
smaller_df_1 = reviews_df[random_array < 1/3]
smaller_df_2 = reviews_df[(random_array >= 1/3) & (random_array < 2/3)]
smaller_df_3 = reviews_df[random_array >= 2/3]

In [51]:
smaller_df_2

,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified
3,13,7435,1773,Malarkey Test,5,easily the best i have ever had. juicy flavor...,2000-03-13T21:15:00Z,2000-03-13T21:15:00Z
5,17,5221,2046,Gay Gilmore ckpt,4,"love it, but without the bean sprouts.",2000-05-08T11:08:00Z,2000-05-08T11:08:00Z
11,25,3431,2046,Gay Gilmore ckpt,5,leeks on a pizza?! it was really delicious. ...,2000-04-07T11:06:00Z,2000-04-07T11:06:00Z
20,46,7226,2594,Catherine Allison,4,This recipe was really easy to follow and was ...,2000-09-12T22:49:08Z,2000-09-12T22:49:08Z
26,54,8476,2509,CAROLYN INCH,5,I made this recipe for 16 coworkers. I double...,2000-10-05T17:40:36Z,2000-10-05T17:40:36Z
...,...,...,...,...,...,...,...,...
1401963,2090323,450339,2002901298,Karen G.,0,Wow! Love Paula Deen's recipes and this is no ...,2020-12-27T16:32:26Z,2020-12-27T16:32:26Z
1401965,2090325,268004,2002826753,Irma K.,5,Absolutely wonderful! The only changes I initi...,2020-12-27T17:41:11Z,2020-12-27T17:41:11Z
1401970,2090331,446285,2002901639,twbahamas,5,I tried it! Messed up by using salted butter a...,2020-12-27T21:17:47Z,2020-12-27T21:17:47Z
1401972,2090334,105865,2002901740,jessica92983,2,I’m not usually a big chili fan but wanted to ...,2020-12-27T23:04:59Z,2020-12-27T23:04:59Z


In [50]:
ReviewsToLoad = smaller_df_2[['RecipeId', 'Rating', 'Review']]

In [52]:
ReviewsToLoad.head()

,RecipeId,Rating,Review
3,7435,5,easily the best i have ever had. juicy flavor...
5,5221,4,"love it, but without the bean sprouts."
11,3431,5,leeks on a pizza?! it was really delicious. ...
20,7226,4,This recipe was really easy to follow and was ...
26,8476,5,I made this recipe for 16 coworkers. I double...


In [53]:
ReviewsToLoad.to_json('ReviewsToLoad.json', orient="records") # change it into a json type

In [12]:
DataToLoad.head()

,RecipeId,Name,TotalTime,Description,RecipeCategory,Keywords,Calories,FatContent,SaturatedFatContent,CholesterolContent,...,ingredients,Calories_PerServing,FatContent_PerServing,SaturatedFatContent_PerServing,CholesterolContent_PerServing,SodiumContent_PerServing,CarbohydrateContent_PerServing,FiberContent_PerServing,SugarContent_PerServing,ProteinContent_PerServing
0,39,Biryani,265,Make and share this Biryani recipe from Food.com.,Chicken Breast,"['Chicken Thigh & Leg', 'Chicken', 'Poultry', ...",1110.7,58.8,16.6,372.8,...,"['saffron', 'milk', 'hot green chili pepper', ...",185.116667,9.8000,2.766667,62.133333,61.400,14.066667,1.5000,3.4000,10.566667
1,41,Carina's Tofu-Vegetable Kebabs,1460,This dish is best prepared a day in advance to...,Soy/Tofu,"['Beans', 'Vegetable', 'Low Cholesterol', 'Wee...",536.1,24.0,3.8,0.0,...,"['extra firm tofu', 'eggplant', 'zucchini', 'm...",268.050000,12.0000,1.900000,0.000000,779.300,32.100000,8.6500,16.0500,14.650000
2,43,Best Blackbottom Pie,140,Make and share this Best Blackbottom Pie recip...,Pie,"['Dessert', 'Weeknight', 'Stove Top', '< 4 Hou...",437.9,19.3,10.9,94.3,...,"['graham cracker crumb', 'sugar', 'butter', 's...",54.737500,2.4125,1.362500,11.787500,33.450,7.250000,0.2250,5.3125,0.875000
3,45,Buttermilk Pie With Gingersnap Crumb Crust,80,Make and share this Buttermilk Pie With Ginger...,Pie,"['Dessert', 'Healthy', 'Weeknight', 'Oven', '<...",228.0,7.1,1.7,24.5,...,"['sugar', 'margarine', 'egg', 'flour', 'salt',...",28.500000,0.8875,0.212500,3.062500,35.225,4.687500,0.0625,3.0875,0.525000
4,47,Butter Pecan Cookies,64,Make and share this Butter Pecan Cookies recip...,Dessert,"['Cookie & Brownie', 'Fruit', 'Nuts', 'Weeknig...",69.0,5.6,1.4,6.3,...,"['butter', 'brown sugar', 'sugar', 'vanilla ex...",69.000000,5.6000,1.400000,6.300000,15.000,4.500000,0.6000,1.6000,0.800000


In [15]:
DataToLoad.to_json('DataToLoad.json', orient="records") # change it into a json type

In [17]:
with open('DataToLoad.json', "r") as f:
    jsonData = json.load(f)
    jsonDatastr = json.dumps(jsonData)


**example  first is the object in the database then it is the data from the dataframe

client.query("""
    with
        raw_data := <json>$data,
    for item in json_array_unpack(raw_data) union (
        insert Recipe {Rating := <float32>item['Rating'], 
        RecipeId := <int32>item['RecipeId'],
        Review := <str>item['Review']
        }
    );
""", data=jsonReviewsstr)

In [37]:
# Approach 1: Using np.array_split()
num_splits = 3
smaller_dfs_array_split = np.array_split(DataToLoad, num_splits)

# Calculate the approximate number of rows for each smaller DataFrame
num_rows = len(DataToLoad)
rows_per_df = num_rows // num_splits

# Approach 1: Using np.array_split()
smaller_dfs_array_split = np.array_split(DataToLoad, num_splits)

# Approach 2: Using DataFrame slicing
smaller_df_1 = DataToLoad[:rows_per_df]
smaller_df_2 = DataToLoad[rows_per_df: 2 * rows_per_df]
smaller_df_3 = DataToLoad[2 * rows_per_df:]

# Approach 3: Using np.random.rand()
random_array = np.random.rand(len(DataToLoad))

smaller_df_1 = DataToLoad[random_array < 1/3]
smaller_df_2 = DataToLoad[(random_array >= 1/3) & (random_array < 2/3)]
smaller_df_3 = DataToLoad[random_array >= 2/3]

In [38]:
DataToLoad1 = smaller_df_1
DataToLoad2 = smaller_df_2
DataToLoad3 = smaller_df_3

In [39]:
DataToLoad1.to_json('DataToLoad1.json', orient="records")
DataToLoad2.to_json('DataToLoad2.json', orient="records")
DataToLoad3.to_json('DataToLoad3.json', orient="records")

In [40]:
with open('DataToLoad1.json', "r") as f:
    jsonData1 = json.load(f)
    jsonData1str = json.dumps(jsonData1)
    

In [41]:
with open('DataToLoad2.json', "r") as f:
    jsonData2 = json.load(f)
    jsonData2str = json.dumps(jsonData2)
    

In [49]:
with open('DataToLoad3.json', "r") as f:
    jsonData3 = json.load(f)
    jsonData3str = json.dumps(jsonData3)
    

In [46]:
# Assuming you have a JSON data string named 'jsonData1str'
data = json.loads(jsonData1str)

# Split the data into smaller batches
batch_size = 100  # Choose an appropriate batch size
num_batches = len(data) // batch_size + (1 if len(data) % batch_size > 0 else 0)

for batch_num in range(num_batches):
    start_idx = batch_num * batch_size
    end_idx = (batch_num + 1) * batch_size
    batch_data = data[start_idx:end_idx]

    # Convert the batch data back to a JSON string
    batch_data_str = json.dumps(batch_data)

    # Perform the insertion for the current batch
    client.query("""
        WITH raw_data := <json>$data
        FOR item IN json_array_unpack(raw_data) UNION (
            INSERT Recipe {
                name := <str>item['Name'],
                RecipeId := <int32>item['RecipeId'],
                TotalTime := <int32>item['TotalTime'],
                Description := <str>item['Description'],
                RecipeCategory := <str>item['RecipeCategory'],
                Keywords := <str>item['Keywords'],
                Calories := <float32>item['Calories'],
                FatContent := <float32>item['FatContent'],
                SaturatedFatContent := <float32>item['SaturatedFatContent'],
                CholestrerolContent := <float32>item['CholesterolContent'],
                SodiumContent := <float32>item['SodiumContent'],
                CarboyhdrateContent := <float32>item['CarbohydrateContent'],
                FiberContent := <float32>item['FiberContent'],
                SugarContent := <float32>item['SugarContent'],
                ProteinContent := <float32>item['ProteinContent'],
                Calories_PerServing := <float32>item['Calories_PerServing'],
                FatContent_PerServing := <float32>item['FatContent_PerServing'],
                SaturatedFatContent_PerServing := <float32>item['SaturatedFatContent_PerServing'],
                CholesterolContent_PerServing := <float32>item['CholesterolContent_PerServing'],
                SodiumContent_PerServing := <float32>item['SodiumContent_PerServing'],
                CarbohydrateContent_PerServing := <float32>item['CarbohydrateContent_PerServing'],
                FiberContent_PerServing := <float32>item['FiberContent_PerServing'],
                SugarContent_PerServing := <float32>item['SugarContent_PerServing'],
                ProteinContent_PerServing := <float32>item['ProteinContent_PerServing'],
                RecipeInstructions := <str>item['RecipeInstructions'],
                Popularity := <str>item['Popularity'],
                ingredient_num := <int32>item['ingredient_num'],
                ingredients := <str>item['ingredients']
            }
        );
    """, data=batch_data_str)

In [47]:
# Assuming you have a JSON data string named 'jsonData1str'
data = json.loads(jsonData2str)

# Split the data into smaller batches
batch_size = 100  # Choose an appropriate batch size
num_batches = len(data) // batch_size + (1 if len(data) % batch_size > 0 else 0)

for batch_num in range(num_batches):
    start_idx = batch_num * batch_size
    end_idx = (batch_num + 1) * batch_size
    batch_data = data[start_idx:end_idx]

    # Convert the batch data back to a JSON string
    batch_data_str = json.dumps(batch_data)

    # Perform the insertion for the current batch
    client.query("""
        WITH raw_data := <json>$data
        FOR item IN json_array_unpack(raw_data) UNION (
            INSERT Recipe {
                name := <str>item['Name'],
                RecipeId := <int32>item['RecipeId'],
                TotalTime := <int32>item['TotalTime'],
                Description := <str>item['Description'],
                RecipeCategory := <str>item['RecipeCategory'],
                Keywords := <str>item['Keywords'],
                Calories := <float32>item['Calories'],
                FatContent := <float32>item['FatContent'],
                SaturatedFatContent := <float32>item['SaturatedFatContent'],
                CholestrerolContent := <float32>item['CholesterolContent'],
                SodiumContent := <float32>item['SodiumContent'],
                CarboyhdrateContent := <float32>item['CarbohydrateContent'],
                FiberContent := <float32>item['FiberContent'],
                SugarContent := <float32>item['SugarContent'],
                ProteinContent := <float32>item['ProteinContent'],
                Calories_PerServing := <float32>item['Calories_PerServing'],
                FatContent_PerServing := <float32>item['FatContent_PerServing'],
                SaturatedFatContent_PerServing := <float32>item['SaturatedFatContent_PerServing'],
                CholesterolContent_PerServing := <float32>item['CholesterolContent_PerServing'],
                SodiumContent_PerServing := <float32>item['SodiumContent_PerServing'],
                CarbohydrateContent_PerServing := <float32>item['CarbohydrateContent_PerServing'],
                FiberContent_PerServing := <float32>item['FiberContent_PerServing'],
                SugarContent_PerServing := <float32>item['SugarContent_PerServing'],
                ProteinContent_PerServing := <float32>item['ProteinContent_PerServing'],
                RecipeInstructions := <str>item['RecipeInstructions'],
                Popularity := <str>item['Popularity'],
                ingredient_num := <int32>item['ingredient_num'],
                ingredients := <str>item['ingredients']
            }
        );
    """, data=batch_data_str)

In [50]:
# Assuming you have a JSON data string named 'jsonData1str'
data = json.loads(jsonData3str)

# Split the data into smaller batches
batch_size = 100  # Choose an appropriate batch size
num_batches = len(data) // batch_size + (1 if len(data) % batch_size > 0 else 0)

for batch_num in range(num_batches):
    start_idx = batch_num * batch_size
    end_idx = (batch_num + 1) * batch_size
    batch_data = data[start_idx:end_idx]

    # Convert the batch data back to a JSON string
    batch_data_str = json.dumps(batch_data)

    # Perform the insertion for the current batch
    client.query("""
        WITH raw_data := <json>$data
        FOR item IN json_array_unpack(raw_data) UNION (
            INSERT Recipe {
                name := <str>item['Name'],
                RecipeId := <int32>item['RecipeId'],
                TotalTime := <int32>item['TotalTime'],
                Description := <str>item['Description'],
                RecipeCategory := <str>item['RecipeCategory'],
                Keywords := <str>item['Keywords'],
                Calories := <float32>item['Calories'],
                FatContent := <float32>item['FatContent'],
                SaturatedFatContent := <float32>item['SaturatedFatContent'],
                CholestrerolContent := <float32>item['CholesterolContent'],
                SodiumContent := <float32>item['SodiumContent'],
                CarboyhdrateContent := <float32>item['CarbohydrateContent'],
                FiberContent := <float32>item['FiberContent'],
                SugarContent := <float32>item['SugarContent'],
                ProteinContent := <float32>item['ProteinContent'],
                Calories_PerServing := <float32>item['Calories_PerServing'],
                FatContent_PerServing := <float32>item['FatContent_PerServing'],
                SaturatedFatContent_PerServing := <float32>item['SaturatedFatContent_PerServing'],
                CholesterolContent_PerServing := <float32>item['CholesterolContent_PerServing'],
                SodiumContent_PerServing := <float32>item['SodiumContent_PerServing'],
                CarbohydrateContent_PerServing := <float32>item['CarbohydrateContent_PerServing'],
                FiberContent_PerServing := <float32>item['FiberContent_PerServing'],
                SugarContent_PerServing := <float32>item['SugarContent_PerServing'],
                ProteinContent_PerServing := <float32>item['ProteinContent_PerServing'],
                RecipeInstructions := <str>item['RecipeInstructions'],
                Popularity := <str>item['Popularity'],
                ingredient_num := <int32>item['ingredient_num'],
                ingredients := <str>item['ingredients']
            }
        );
    """, data=batch_data_str)

In [54]:
with open('ReviewsToLoad.json', "r") as f:
    jsonReviews = json.load(f)
    jsonReviewsstr = json.dumps(jsonReviews)

In [55]:
print(jsonReviewsstr)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [56]:
#ReviewsToLoad = review_DF[['RecipeId', 'Rating', 'Review']] - remove review file was too big

client.query("""
    with
        raw_data := <json>$data,
    for item in json_array_unpack(raw_data) union (
        insert Review {Rating := <float32>item['Rating'], 
        RecipeId := <int32>item['RecipeId'],
        Review := <str>item['Review']
        }
    );
""", data=jsonReviewsstr)

result = client.query("""
    select Review {
    Rating
    }

""")

print(result)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [26]:
ReviewsToLoad = reviews_df[['RecipeId','Review']]

In [27]:
ReviewsToLoad.head()

,RecipeId,Review
0,992,better than any you can get at a restaurant!
1,4384,"I cut back on the mayo, and made up the differ..."
2,4523,i think i did something wrong because i could ...
3,7435,easily the best i have ever had. juicy flavor...
4,44,An excellent dish.


In [28]:
ReviewsToLoad.to_json('ReviewsToLoad.json', orient="records") # change it into a json type

In [29]:
with open('ReviewsToLoad.json', "r") as f:
    jsonReviews = json.load(f)
    jsonReviewsstr = json.dumps(jsonReviews)

In [35]:
#ReviewsToLoad = review_DF[['RecipeId', 'Rating', 'Review']] - remove review file was too big

client.query("""
    with
        raw_data := <json>$data,
    for item in json_array_unpack(raw_data) union (
        update Review {RecipeId := <int32>item['RecipeId'],
        insert Review {Review := <str>item['Review']
        
            }
        }
    );
""", data=jsonReviewsstr)

result = client.query("""
    select Review {
    Review
    }

""")

print(result)

EdgeQLSyntaxError: Unexpected keyword 'insert'
   ┌─ query:6:9
   │ 
 6 │           insert Review {Review := <str>item['Review']
   │           ^^^^^^ Use a different identifier or quote the name with backticks: `insert`

In [33]:
client.query("""
    WITH
        raw_data := <json>$data
    # Define a SET OF Review objects with corresponding Review text for each item in the JSON data.
    # We use the "FOR" loop to unpack the JSON data and create Review objects.
    # Note: Make sure to replace 'Review' with the actual object type name in your schema.
    INSERT { 
        FOR item IN json_array_unpack(raw_data)
        UNION (
            Review {
                Review := <str>item['Review'],
                RecipeId := <int32>item['RecipeId']
            }
        )
    };
""", data=jsonReviewsstr)

# Now, after inserting the reviews, you can retrieve the 'Review' objects.
result = client.query("""
    SELECT Review {
        Review
    };
""")
print(result)

EdgeQLSyntaxError: Unexpected keyword 'FOR'
   ┌─ query:8:9
   │ 
 8 │           FOR item IN json_array_unpack(raw_data)
   │           ^^^ Use a different identifier or quote the name with backticks: `FOR`